In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv("Variables/Faillissementen98-10.csv", sep = ";", index_col = 0, skiprows = 3,skipfooter = 1, engine='python')

df = df.drop("Regio's" ,axis = 0)

for i in df.index.values:
    df = df.rename({i:i.replace(" (PV)","")},axis = 0)
# display(df)


dfprov = pd.read_csv("Provincies2020", sep = ";", skiprows = 3,skipfooter = 1,engine='python')
dfprov = dfprov.drop(0,axis = 0)
dfprov = dfprov.rename({"Unnamed: 0":"Regionaam","Codes en namen van gemeenten|Code":"Regiocode","Lokaliseringen van gemeenten|Provincies|Naam":"Provincie"},axis = 1)



In [16]:
bankr10 = pd.DataFrame()
for index, provincie in enumerate(dfprov["Provincie"]):
    if provincie == "Fryslân":
        provincie = "Friesland"

    bankr10[dfprov.iloc[index,1]] = df.loc[provincie]
bankr10 = bankr10.T
# ## ADDING Municipality Littenseradeel, which was splitted in different municipalities in 2018
bankr10.loc["GM0140"] = df.loc["Friesland"].tolist()

newdict = {}
for i in bankr10:
    newdict[i] = "Bankruptcies{}".format(i)
    
bankr10 = bankr10.rename(newdict, axis =1 )

for index, code in enumerate(bankr10.index.values):
    bankr10 = bankr10.rename({code: code.replace("M","")})
    

bankr10 = bankr10.rename({"index":"Regiocode"},axis =1)
bankr10 = bankr10.astype(int)
display(bankr10)

,Bankruptcies1998,Bankruptcies1999,Bankruptcies2000,Bankruptcies2001,Bankruptcies2002,Bankruptcies2003,Bankruptcies2004,Bankruptcies2005,Bankruptcies2006,Bankruptcies2007,Bankruptcies2008,Bankruptcies2009,Bankruptcies2010
G1680,58,57,74,90,104,154,156,127,119,103,145,227,193
G0358,477,421,414,648,680,872,834,864,715,597,706,1177,1025
G0197,271,275,335,371,446,616,609,600,512,414,413,862,726
G0059,59,49,76,91,108,148,134,135,109,92,98,194,151
G0482,601,516,595,721,940,1141,1162,1117,949,725,721,1359,1325
...,...,...,...,...,...,...,...,...,...,...,...,...,...
G0301,271,275,335,371,446,616,609,600,512,414,413,862,726
G1896,133,139,140,237,248,316,339,296,270,239,236,456,470
G0642,601,516,595,721,940,1141,1162,1117,949,725,721,1359,1325
G0193,133,139,140,237,248,316,339,296,270,239,236,456,470


In [17]:
for year in range(1998,2010):
    bankr10["avgbankrchange{}".format(year+1)] = (((bankr10["Bankruptcies{}".format(year+1)]) - bankr10["Bankruptcies{}".format(year)] )/bankr10["Bankruptcies{}".format(year)])*100
bankr10 = bankr10.reset_index()
bankr10 = bankr10.rename({"index":"Regiocode"}, axis = 1)
# display(bankr10)
electionperiods = [[1999,2002,"9802"],[2002,2003,"0203"],[2003,2006,"0306"],[2006,2010,"0610"]]
dfBankr10 = pd.DataFrame()


for i in electionperiods:
    startyear = i[0]+1
    avg = bankr10[["Regiocode","avgbankrchange{}".format(startyear)]]
    for j in range(i[0]+2,i[1]+1):
        avg = avg.merge(bankr10[["avgbankrchange{}".format(j),"Regiocode"]],how = "right", on = "Regiocode")
    dfBankr10["avgbankrchangeelection{}".format(i[2])] = avg.iloc[:,1:].mean(axis = 1)



dfBankr10["Regiocode"] = bankr10["Regiocode"]



In [18]:
dfBankr17 = pd.read_csv("Variables/Definitiveversions/Bank10-17_election.csv", index_col = 0)
dfBankruptcies_election = dfBankr10.merge(dfBankr17, on = ["Regiocode"] ,how = "outer")


,avgbankrchangeelection1012,avgbankrchangeelection1217,Regiocode
0,13.509308,-15.022510,G1680
1,5.633608,-11.672040,G0358
2,18.229700,-15.820925,G0197
3,39.895833,-14.858751,G0059
4,1.923668,-11.812285,G0482
...,...,...,...
351,18.229700,-15.820925,G0301
352,12.553296,-14.644288,G1896
353,1.923668,-11.812285,G0642
354,12.553296,-14.644288,G0193


,avgbankrchangeelection9802,avgbankrchangeelection0203,avgbankrchangeelection0306,avgbankrchangeelection0610,Regiocode
0,22.333913,48.076923,-7.863418,17.226268,G1680
1,19.932434,28.235294,-6.002015,13.888538,G0358
2,17.593361,38.116592,-5.760288,18.389318,G0197
3,31.173401,37.037037,-9.324150,16.679911,G0059
4,22.287009,21.382979,-5.690810,15.457713,G0482
...,...,...,...,...,...
351,17.593361,38.116592,-5.760288,18.389318,G0301
352,24.882163,27.419355,-4.729890,20.888451,G1896
353,22.287009,21.382979,-5.690810,15.457713,G0642
354,24.882163,27.419355,-4.729890,20.888451,G0193
